In [1]:
# Importing standard packages for data exploration and processing.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


# Our data has no header row.
data = pd.read_csv('data/kaggle/1_steam-200k.csv', header=None)
data.head()

,0,1,2,3,4
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       200000 non-null  int64  
 1   1       200000 non-null  object 
 2   2       200000 non-null  object 
 3   3       200000 non-null  float64
 4   4       200000 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


In [3]:
data.describe()

,0,3,4
count,2.000000e+05,200000.000000,200000.0
mean,1.036559e+08,17.874384,0.0
std,7.208074e+07,138.056952,0.0
min,5.250000e+03,0.100000,0.0
25%,4.738420e+07,1.000000,0.0
50%,8.691201e+07,1.000000,0.0
75%,1.542309e+08,1.300000,0.0
max,3.099031e+08,11754.000000,0.0


In [4]:
# The last column is useless.
data.drop(4, axis=1, inplace=True)

# Current column names are not very informative, are they?
header = ['Player_id', 'Game', 'Action', 'Amount']
data.columns = header
data.head()

,Player_id,Game,Action,Amount
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [5]:
# We not only need to aggregate the data to a game level but also store number of purchases and hours of play separately.
summary = data.pivot_table(index='Game', columns='Action', values='Amount', aggfunc='sum')
summary.reset_index(inplace=True)
summary.rename_axis(None, axis=1, inplace=True)
summary.head()

,Game,play,purchase
0,007 Legends,0.7,1.0
1,0RBITALIS,1.2,3.0
2,1... 2... 3... KICK IT! (Drop That Beat Like a...,20.0,7.0
3,10 Second Ninja,5.9,6.0
4,"10,000,000",3.6,1.0


In [6]:
# It would make more sense to have the 'Purchase' column before 'Play'.
summary.columns = ['Game', 'Play', 'Purchase']
summary = summary[['Game', 'Purchase', 'Play']]
summary.head()

,Game,Purchase,Play
0,007 Legends,1.0,0.7
1,0RBITALIS,3.0,1.2
2,1... 2... 3... KICK IT! (Drop That Beat Like a...,7.0,20.0
3,10 Second Ninja,6.0,5.9
4,"10,000,000",1.0,3.6


In [7]:
summary.describe()

,Purchase,Play
count,5155.000000,3600.000000
mean,25.123375,957.046056
std,101.572586,17930.737469
min,1.000000,0.100000
25%,2.000000,3.100000
50%,5.000000,15.100000
75%,18.000000,97.575000
max,4841.000000,981684.600000


Quite a lot of games that were bought but never played in our data, is it not? 5155 games have at least one purchase while only 3600 games have at least one person that played them. Since we are not really interested in such unpopular games, let us put some minimum qualifying restrictions.

In [8]:
summary = summary[(summary['Purchase'] >= 10) & (summary['Play'] >= 100)]
summary.reset_index(drop=True, inplace=True)
summary.head()

,Game,Purchase,Play
0,3DMark,18.0,240.8
1,7 Days to Die,103.0,4075.5
2,A.V.A - Alliance of Valiant Arms,63.0,109.1
3,APB Reloaded,262.0,9237.2
4,ARK Survival Evolved,163.0,13593.1


In [9]:
summary.describe()

,Purchase,Play
count,783.000000,783.000000
mean,111.197957,4303.685185
std,237.255413,38279.790989
min,10.000000,100.300000
25%,29.000000,200.100000
50%,57.000000,468.500000
75%,113.500000,1418.200000
max,4841.000000,981684.600000


783 games, not bad. We need to remember that the data used here is nowhere near the actual popularity of the games as it only contains 200,000 actions and is rather old. Moreover, some games might not be properly represented due to the small scale of dataset. However, we can presume that at least for the most popular games that should not be the case. And let us create a shortlist of games with stricter qualifying requirements.

In [10]:
shortlist = summary[(summary['Purchase'] >= 100) & (summary['Play'] >= 1000)]
shortlist.reset_index(drop=True, inplace=True)
shortlist.head()

,Game,Purchase,Play
0,7 Days to Die,103.0,4075.5
1,APB Reloaded,262.0,9237.2
2,ARK Survival Evolved,163.0,13593.1
3,AdVenture Capitalist,222.0,6067.7
4,Age of Empires II HD Edition,238.0,6858.5


In [11]:
shortlist.describe()

,Purchase,Play
count,155.000000,155.000000
mean,319.477419,18460.578710
std,461.029598,84669.048622
min,100.000000,1002.800000
25%,142.000000,1816.950000
50%,207.000000,3581.900000
75%,313.000000,8840.300000
max,4841.000000,981684.600000


155 games, now that should be much easier to look at and analyse one by one.